# Required Packages

- jupyterlab>=4.4.1
- google-analytics-admin>=0.24.0
- google-api-python-client>=2.168.0
- google-auth>=2.39.0
- google-auth-oauthlib>=1.2.2

In [ ]:
import os.path
import google.auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import google.analytics.admin as gaa
from google.api_core import exceptions

In [ ]:
# set config constants

TARGET_PROPERTY_ID = "451062722"
SOURCE_SHEET_ID = "11mCeNbPAfH3JWm_cj07BD3_m-lfr6R1ZVxKwcEvIBhw"
SOURCE_RANGE = "A:G"

# Turn IDs into expected name strings

TARGET_PROP_NAME = "properties/" + TARGET_PROPERTY_ID

In [ ]:
creds = None
SCOPES = [
        "https://www.googleapis.com/auth/analytics.edit",
        "https://www.googleapis.com/auth/analytics.readonly",
        "https://www.googleapis.com/auth/spreadsheets",
    ]
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(google.auth.transport.requests.Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=1234)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

In [ ]:
# Build sheets api client
SHEETS_CLIENT = build("sheets", "v4", credentials=creds)

# read in spreadsheet data
sheet = SHEETS_CLIENT.spreadsheets()
result = (
    sheet.values()
    .get(spreadsheetId=SOURCE_SHEET_ID, range=SOURCE_RANGE)
    .execute()
)
values = result.get("values", [])

# remove header row
rows = values[1:]
rows

In [ ]:
# create GA admin api client
GAA_CLIENT = gaa.AnalyticsAdminServiceClient(credentials=creds)

# define a function to create a dimension
def create_dim(parameter_name, display_name, scope, description):
    new_dim = gaa.CustomDimension()
    new_dim.parameter_name = parameter_name
    new_dim.display_name = display_name
    new_dim.scope = scope.upper()
    new_dim.description = description

    request = gaa.CreateCustomDimensionRequest(
        custom_dimension = new_dim,
        parent = TARGET_PROP_NAME, 
    )
    response = GAA_CLIENT.create_custom_dimension(request=request)
    
    return response

# define a function to create a metric
def create_metric(parameter_name, display_name, measurement_unit, description):
    new_metric = gaa.CustomMetric()
    new_metric.parameter_name = parameter_name
    new_metric.display_name = display_name
    new_metric.scope = "EVENT"
    new_metric.measurement_unit = measurement_unit.upper()
    new_metric.description = description

    request = gaa.CreateCustomMetricRequest(
        custom_metric = new_metric,
        parent = TARGET_PROP_NAME, 
    )
    response = GAA_CLIENT.create_custom_metric(request=request)
    
    return response


# define a function to update the spreadsheet 'Configured' column
def mark_row_congfigured(i):
    # update the configured column
    SHEETS_CLIENT.spreadsheets().values().update(
        spreadsheetId=SOURCE_SHEET_ID,
        range="H" + str(i),
        valueInputOption="USER_ENTERED",
        body={"values": [["x"]]},
    ).execute()
    return
  

In [ ]:
# define a function to create a dimension or metric
def create_dim_or_metric(row,i):

    # check if the row is short and fix it
    while len(row) <= 6:
        row.append("")

    # unpack the row
    parameter_name, stream, dimension_name, metric_name, scope_or_unit, type, required, all_hits, configured, valid, description = row

    # update the user
    print(f"Processing row {i}: {parameter_name}")

    if len(required) == 0:
        print(f"Skipping {parameter_name}: it is not marked as required.")
    
    elif type.upper() == "BUILT-IN":
        mark_row_congfigured(i)
        print(f"Skipping {parameter_name}: it is a built-in dimension or metric. Marked row {i} as configured")
        return

    # check if the row is valid
    elif (dimension_name == "" and metric_name == "") or (len(dimension_name) > 0 and len(metric_name) > 0):
        print(f"Invalid row {parameter_name}: must have either a dimension or metric name, not both.")
        return

    elif len(dimension_name) > 0:
        scope = scope_or_unit.upper()
        try:
            response = create_dim(parameter_name, dimension_name, scope, description)
            mark_row_congfigured(i)
            print(f"Created dimension: {response.display_name} and marked row {i} as configured.")
        except exceptions.AlreadyExists:
            print(f"Dimension already exists: {dimension_name}")
            
    else:
        measurement_unit = scope_or_unit.upper()
        try:
            response = create_metric(parameter_name, metric_name, measurement_unit, description)
            mark_row_congfigured(i)
            print(f"Created metric: {response.display_name} and marked row {i} as configured.")
        except exceptions.AlreadyExists:
            print(f"Metric already exists: {metric_name}")
      


In [ ]:
i=1
for row in rows:
    i+= 1
    create_dim_or_metric(row, i)